In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('data/train_sample.csv')

In [39]:
data[data['is_attributed'] == 1][['ip','app']].value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'